In [ ]:
# Transformer
# Seq2Seq_attention_번역_Q

In [ ]:
# 3. 실습1 - Seq2Seq 모델을 학습시켜서 번역기 만들기

# Encoder, Decoder 를 만든 뒤 Seq2Seq 모델로 조립한 후, 프랑스어와 영어로 매칭된 사전을 학습시켜 번역기 만들기

import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# 데이터셋 불러오기
lines = pd.read_csv(
  'fra.txt',
  names=['src', 'tar', 'lic'],
  sep='\t' # \t 을 기준으로 컬럼 분리
)
del lines['lic'] # license 컬럼이 존재하는 데이터셋의 경우 필요없기 때문에 삭제
print(f"전체 샘플의 개수: {len(lines)}")

# 데이터셋 슬라이싱
lines = lines.loc[:, 'src':'tar']
lines = lines[:30000]
display(lines.sample(10)) # 전체 DataFrame에서 랜덤하게 (n)개의 행을 추출

# seq2seq 모델 학습을 위한 특수 토큰 삽입
# \t = 시작(start-of-sequence) 토큰
# \n = 시작(end-of-sequence) 토큰
lines.tar = lines.tar.apply(lambda x: '\t' + x + '\n')
display(lines.sample(10))

In [ ]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src:
  for char in line:
    src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print(f"source 문장의 char집합: {src_vocab_size}")
print(f"target 문장의 char집합: {tar_vocab_size}\n")

src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(f"{src_vocab[:10]}")
print(f"{tar_vocab[:10]}\n")

In [ ]:
# 문장 인덱스화

src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(f"{src_to_index}")
print(f"{tar_to_index}\n")

encoder_input = []
for line in lines.src:
  encoded_line = []
  for char in line:
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print(f"원래 source 문장:\n{lines.src[:3]}")
print(f"인코더 입력 시퀀스: {encoder_input[:3]}\n")

decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print(f"디코더 입력 시퀀스: {decoder_input[:3]}\n")

decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print(f"디코더 정답 시퀀스: {decoder_target[:3]}\n")

In [ ]:
# 제로 패딩을 적용하기 위한 최대 길이
# + One-Hot Encoding

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(f"source 문장의 최대 길이: {max_src_len}")
print(f"source 문장의 최대 길이: {max_tar_len}\n")

encoder_input = pad_sequences(
  encoder_input,
  maxlen=max_src_len,
  padding='post'
)
decoder_input = pad_sequences(
  decoder_input,
  maxlen=max_tar_len,
  padding='post'
)
decoder_target = pad_sequences(
  decoder_target,
  maxlen=max_tar_len,
  padding='post'
)
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [ ]:
# 모델 정의

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(256, return_state=True) # Encoder는 return_state를 설정한 LSTM 선언
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c] # LSTM 은 상태가 두개 (은닉 상태와 셀 상태)

decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(256, return_state=True, return_sequences=True) # Decoder는 return_state와 return_sequences를 설정한 LSTM 선언
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states) # Decoder에게 Encoder의 은닉 상태, 셀 상태를 전달

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
seq2seq_outputs = decoder_softmax_layer(decoder_outputs)

model = Model(
  [encoder_inputs, decoder_inputs],
  seq2seq_outputs
)
model.compile(
  optimizer='rmsprop',
  loss='categorical_crossentropy'
)
model.summary()

early_stop = EarlyStopping(
  monitor='val_loss',
  patience=5
)
history = model.fit(
  x=[encoder_input, decoder_input],
  y=decoder_target,
  batch_size=64,
  epochs=50,
  validation_split=0.2,
  callbacks=[early_stop]
)

In [ ]:
# 추론용 인코더, 디코더 모델 재정의

encoder_model = Model(
  inputs=encoder_inputs,
  outputs=encoder_states
)

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용
# decoder_lstm 을 사용하면서 state_h, state_c 같이 저장하기
decoder_outputs_inf, state_h, state_c = decoder_lstm(
  decoder_inputs,
  initial_state=decoder_states_inputs
)

# 훈련 과정에서와 달리 LSTM 의 리턴하는 은닉 상태와 셀 상태를 버리지 않음
decoder_states = [state_h, state_c]
decoder_probs = decoder_softmax_layer(decoder_outputs_inf)
decoder_model = Model(
  inputs=[decoder_inputs] + decoder_states_inputs,
  outputs=[decoder_probs] + decoder_states
)

In [ ]:
# 예측값 디코딩

index_to_tar = dict((i, char) for char, i in tar_to_index.items())

def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)
  
  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.
  
  stop_condition = False
  decoded_sentence = ""
  
  # stop_condition 이 True 가 될 때까지 루프 반복
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]
    
    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char
    
    # <eos>에 도달하거나 최대 길이를 넘으면 중단
    if(sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
      stop_condition = True
      
    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.
    
    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]
  
  return decoded_sentence

In [ ]:
# 예측

for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(f"입력 문장: {lines.src[seq_index]}")
  print(f"정답 문장: {lines.tar[seq_index][1:len(lines.tar[seq_index])]}")
  print(f"번역 문장: {decoded_sentence[0:len(decoded_sentence)]}")